<a href="https://colab.research.google.com/github/CasCard/RIG-Works/blob/main/Copy_of_image_procssing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
from google.colab import drive
from google.colab.patches import cv2_imshow

In [2]:
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [3]:
video1 = '/content/gdrive/My Drive/auv_files/redrodtrim.mp4'

In [5]:
import cv2
from skimage.color import rgb2hsv,hsv2rgb
import numpy as np


def  HSVStretching(sceneRadiance):
    sceneRadiance = np.uint8(sceneRadiance)
    height = len(sceneRadiance)
    width = len(sceneRadiance[0])
    img_hsv = rgb2hsv(sceneRadiance)
    h, s, v = cv2.split(img_hsv)
    img_s_stretching = global_stretching(s, height, width)
    img_v_stretching = global_stretching(v, height, width)

    labArray = np.zeros((height, width, 3), 'float64')
    labArray[:, :, 0] = h
    labArray[:, :, 1] = img_s_stretching
    labArray[:, :, 2] = img_v_stretching
    img_rgb = hsv2rgb(labArray) * 255

    # img_rgb = np.clip(img_rgb, 0, 255)

    return img_rgb

In [6]:
import numpy as np

def sceneRadianceRGB(sceneRadiance):

    sceneRadiance = np.clip(sceneRadiance, 0, 255)
    sceneRadiance = np.uint8(sceneRadiance)

    return sceneRadiance



In [7]:
import numpy as np

def global_stretching(img_L,height, width):
    I_min = np.min(img_L)
    I_max = np.max(img_L)
    I_mean = np.mean(img_L)


    # print('I_min',I_min)
    # print('I_max',I_max)
    # print('I_max',I_mean)

    array_Global_histogram_stretching_L = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            p_out = (img_L[i][j] - I_min) * ((1) / (I_max - I_min))
            array_Global_histogram_stretching_L[i][j] = p_out

    return array_Global_histogram_stretching_L



In [8]:
import numpy as np

def histogram_r(r_array,height, width):
    length = height * width
    R_rray = []
    for i in range(height):
        for j in range(width):
            R_rray.append(r_array[i][j])
    R_rray.sort()
    I_min = int(R_rray[int(length / 500)])
    I_max = int(R_rray[-int(length / 500)])
    array_Global_histogram_stretching = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_min
            elif (r_array[i][j] > I_max):
                p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 255
            else:
                p_out = int((r_array[i][j] - I_min) * ((255 - I_min) / (I_max - I_min)))+ I_min
                array_Global_histogram_stretching[i][j] = p_out
    return (array_Global_histogram_stretching)

def histogram_g(r_array,height, width):
    length = height * width
    R_rray = []
    for i in range(height):
        for j in range(width):
            R_rray.append(r_array[i][j])
    R_rray.sort()
    I_min = int(R_rray[int(length / 500)])
    I_max = int(R_rray[-int(length / 500)])
    array_Global_histogram_stretching = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 255
            else:
                p_out = int((r_array[i][j] - I_min) * ((255) / (I_max - I_min)) )
                array_Global_histogram_stretching[i][j] = p_out
    return (array_Global_histogram_stretching)

def histogram_b(r_array,height, width):
    length = height * width
    R_rray = []
    for i in range(height):
        for j in range(width):
            R_rray.append(r_array[i][j])
    R_rray.sort()
    I_min = int(R_rray[int(length / 500)])
    I_max = int(R_rray[-int(length / 500)])
    array_Global_histogram_stretching = np.zeros((height, width))
    for i in range(0, height):
        for j in range(0, width):
            if r_array[i][j] < I_min:
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = 0
            elif (r_array[i][j] > I_max):
                # p_out = r_array[i][j]
                array_Global_histogram_stretching[i][j] = I_max
            else:
                p_out = int((r_array[i][j] - I_min) * ((I_max) / (I_max - I_min)))
                array_Global_histogram_stretching[i][j] = p_out
    return (array_Global_histogram_stretching)

def stretching(img):
    height = len(img)
    width = len(img[0])
    img[:, :, 2] = histogram_r(img[:, :, 2], height, width)
    img[:, :, 1] = histogram_g(img[:, :, 1], height, width)
    img[:, :, 0] = histogram_b(img[:, :, 0], height, width)
    return img




In [9]:
import numpy as np


def cal_equalisation(img,ratio):
    Array = img * ratio
    Array = np.clip(Array, 0, 255)
    return Array


def RGB_equalisation(img):
    i =0
    img = np.float32(img)
    avg_RGB = []
    for i in range(3):
        avg = np.mean(img[:,:,i])
        avg_RGB.append(avg)
    # print('avg_RGB',avg_RGB)
    a_r = avg_RGB[0]/avg_RGB[2]
    a_g =  avg_RGB[0]/avg_RGB[1]
    ratio = [0,a_g,a_r]
    for i in range(1,3):
        img[:,:,i] = cal_equalisation(img[:,:,i],ratio[i])
    return img






In [14]:
import os
import numpy as np
import cv2
import natsort
import xlwt
import datetime


np.seterr(over='ignore')
if __name__ == '__main__':
    pass

# folder = "/content/gdrive/My Drive"

# # path = sett
# pathinp = folder + "/train_input_870"
# pathout = folder + "/OutputImages"
# filesinp = os.listdir(pathinp)
# filesoup = os.listdir(pathout)
# length = len(filesinp)
# for i in range(length):
#     imginp = filesinp[i]
#     imgdir = pathinp + "/" + imginp
#     print(filesinp[i])
#     print('********    file   ********',filepath)
#     # img = cv2.imread('InputImages/' + file)
#     import timeit
#     start = timeit.default_timer()
#     img = cv2.imread(imgdir)
#     print(len(img))
#     #img = cv2.resize(img,(256,256))
#     #cv2_imshow(img)
#     print(len(img))
#     sceneRadiance = RGB_equalisation(img)
#     sceneRadiance = stretching(sceneRadiance)
#     # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
#     sceneRadiance = HSVStretching(sceneRadiance)
#     sceneRadiance = sceneRadianceRGB(sceneRadiance)
#     cv2.imwrite('/content/gdrive/My Drive/OutputImages/'+ "oup" + imginp,sceneRadiance)
#     #cv2_imshow(sceneRadiance)
#     stop = timeit.default_timer()
#     print('Time: ', stop - start)  
cap = cv2.VideoCapture(video1)
ret,frame=cap.read()
count=0
out = cv2.VideoWriter('/content/gdrive/My Drive/auv_files/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (frame.shape[1], frame.shape[0]))
while cap.isOpened():
    import timeit
    start = timeit.default_timer()
    ret, frame = cap.read()

    # check for successfulness of cap.read()
    if not ret: break

    sceneRadiance = RGB_equalisation(frame)
    sceneRadiance = stretching(sceneRadiance)
    # # cv2.imwrite(folder + '/OutputImages/' + Number + 'Stretched.jpg', sceneRadiance)
    sceneRadiance = HSVStretching(sceneRadiance)
    sceneRadiance = sceneRadianceRGB(sceneRadiance)
    # Save the video
    out.write(sceneRadiance)
    stop = timeit.default_timer()
    print('Time: ', stop - start ,count)
    # cv2_imshow(image) # Note cv2_imshow, not cv2.imshow

    # cv2.waitKey()

# cv2.destroyAllWindows()
cap.release()
out.release()

Time:  8.044928092000191 0
Time:  7.992605076000018 0
Time:  8.125375092000013 0
Time:  7.985296550000385 0
Time:  8.014792862000377 0
Time:  7.950836443999833 0
Time:  7.9955910589997075 0
Time:  7.992386839999654 0
Time:  7.944422223999936 0
Time:  7.898844772999837 0
Time:  7.906647084999804 0
Time:  7.985535965000054 0
Time:  8.094190843999968 0
Time:  7.993717998999728 0
Time:  7.854865010999674 0
Time:  7.970710806999705 0
Time:  8.020669473999988 0
Time:  7.87994498199987 0
Time:  7.895990214999983 0
Time:  7.825953736999963 0
Time:  7.857503327999893 0
Time:  7.964539208000133 0
Time:  7.985065561000283 0
Time:  7.93523476900009 0
Time:  7.882752797000194 0
Time:  7.988967957999648 0
Time:  7.825437544999659 0
Time:  7.937531543000205 0
Time:  7.907727772999806 0
Time:  7.94863898199992 0
Time:  7.921197500000289 0
Time:  7.909699623000051 0
Time:  8.029812733999734 0
Time:  7.898859724999966 0
Time:  7.816607775000193 0
Time:  7.807439960000011 0
Time:  8.069470637999984 0
Tim

In [15]:
cap.release()
out.release()